In [ ]:
import lichess.api
from pprint import pprint
from datetime import datetime
import os

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter

# working dir should be chessanalytics/src when the notebook starts
while(not os.getcwd().endswith('chess-analytics')):
    os.chdir('..')
print('Working dir: ' + os.getcwd())

name = 'carequinha'
user = lichess.api.user(name)

In [ ]:
# %reload_ext autoreload
# %autoreload 2

In [ ]:
from src.game import ChessGame
from src.get_games import download_games
# download_games('carequinha',
#                         is_rated=True)
# download games if they don't exist
# games = load_games('resources/PGN_database')
games = ChessGame.load_pgn_file('src/resources/carequinha.pgn')

In [ ]:
i=5000
print(len(games))

print(games[i].__str__())
print(games[i].result)
print(games[i].get_result(name))
print(list(map(lambda value: value.name, games[i].players.values())))

a = games[i].moves

print(a)
print(a[0::2])
print(games[i].clocks)

In [ ]:
hours = list(map(lambda game: game.utctime.split(':')[0], games))
data_list = list(map(lambda s: int(s), hours))
list_set = set(data_list)
list_set.add(max(list_set)+1)
plt.hist(data_list, bins = list(list_set), edgecolor='blue')
plt.show()

In [ ]:
def get_result_list(game_list):
    list_result = list(map(lambda game: game.get_result(name), game_list))
    return {el: list_result.count(el) for el in set(list_result)}

def parse_time(game):
    year = game.date.split('.')[0]
    month = game.date.split('.')[1]
    return int(year) + int(month)/12

def plot_histogram(data_list, bin_list = None, color = 'green'):
    if bin_list == None:
        bin_list = list(set(data_list))
        bin_list.sort()
        bin_list.append(2*bin_list[-1] - bin_list[-2])
    vertical_lines = set([int(i) for i in bin_list])
    plt.hist(data_list, bins = bin_list, edgecolor='blue', color=color)
    for line in vertical_lines:
        plt.axvline(x=line, color='green')
    return None

def accummulator_list(base_list):
    accum_list = []
    partial_list = []
    for l in data_list:
        accum_list = accum_list + l
    partial_list.append(accum_list)
    return partial_list
    
results = ['WIN','DRAW','LOSS']
colors = ['green','orange','red']

split_game_list = [list(filter(lambda game: game.get_result(name) == el, games)) for el in results]
data_list = list(map(lambda game_list: list(map(lambda game: parse_time(game),game_list)),split_game_list))

data = accummulator_list(data_list)

grouped_plot_properties = [[x,y] for x,y in zip(partial_list, colors)]
grouped_plot_properties.reverse()

plt.figure(figsize=(8,6))
for data, color in grouped_plot_properties:
    plot_histogram(data, bin_list = None, color = color)
plt.show()

In [ ]:
from functools import reduce
def split_list(pattern, base_list):
    values = set([pattern(el) for el in base_list])
    return [[y for y in base_list if pattern(y) == x]for x in values]

def count(pattern, base_list):
    splited_list = split_list((lambda game: pattern(game)), base_list)
    return {pattern(sublist[0]): len(sublist) for sublist in splited_list}

def proportion(a_list, b_list):
    return [float(x)/float(y) for x,y in zip(a_list,b_list)]

def merge_dict(pattern, dict1, dict2):
    dict3 = {**dict1, **dict2}
    for k, v in dict3.items():
        if k in dict1 and k in dict2:
            dict3[k] = pattern(dict1[k], dict2[k])
            
split_game_list = {el: list(filter(lambda game: game.get_result(name) == el, games)) for el in results}
game_counts_list = {k: count(parse_time, l) for (k,l) in split_game_list.items()}

print(game_counts_list)
print(game_counts_list['WIN'])
totals = list(reduce(lambda list_a,list_b : [x+y for x,y in zip(list_a,list_b)] ,game_counts_list))

game_proportion_list = [proportion(l, totals) for l in game_counts_list]
x_data_list = list(set([parse_time(el) for el in l]) for (k,l) in split_game_list.items())
#print(game_proportion_list)
#total_split_count = count(parse_time, games)
#split_win_count = count(parse_time, list_won_games)
#split_windraw_count = count(parse_time, list_wondraw_games)

#total_proportion_list = proportion(split_count,split_count)
#win_proportion_list = proportion(split_win_count,split_count)
#windraw_proportion_list = proportion(split_windraw_count,split_count)
print(len(x_data_list))
for i in range(len(x_data_list)):
    print("game_list length: ", len(game_proportion_list[i]))
    print("x_data length: ", len(x_data_list[i]))
plt.figure(figsize=(8,6))
for lst, x_el in zip(game_proportion_list,x_data_list):
    plt.bar(list(x_el), lst, width=1/12, color = 'red')
#plt.bar(x_data, game_proportion_list[0], width=1/12, color = 'red')
#plt.bar(x_data, windraw_proportion_list, width=1/12, color = 'orange')
#plt.bar(x_data, win_proportion_list, width=1/12, color = 'green')
plt.axhline(y=0.5, color='blue')
plt.show()

In [ ]:
import chess
def print_board_at_move(move_list,
                        move_num: int,
                        player: str):
#move_list = (games[1]['moves'].split())
#move_num = 2
#player = 'black'#'white'
    board = chess.Board()
    board.reset()
    if player is 'white' or player is 'w':
        player_move = 0
    elif player is 'black' or player is 'b':
        player_move = 1
    else:
        raise Exception('No proper input for player.')
    for index in range(2*(move_num-1) + player_move):
        move = move_list.pop(0)
        board.push_san(move)
    return board

In [ ]:
board = print_board_at_move(move_list = (games[0].moves), move_num = 1, player = 'b')
board

In [ ]:
from matplotlib.pyplot import plot
plot([0,1],[0,2])
print(len(games))